# 状況別得点期待値

In [1]:
import pandas as pd
import numpy as np
import re
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from itertools import product
from scipy import stats

%matplotlib inline
plt.style.use('ggplot')
#plt.style.use('seaborn-pastel')
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook as tq

sys.path.append("..")
from utils.util_functions import *

In [2]:
event_df = make_df(data_dir=DETAIL_DATA_DIR_MLB)
df_2017 = make_df(end=1944417, data_dir=DETAIL_DATA_DIR_MLB)
df_2018 = make_df(start=1944417, data_dir=DETAIL_DATA_DIR_MLB)
files = os.listdir(DETAIL_DATA_DIR_MLB)[1:]
names = pd.Series(files).apply(lambda x: int(x.split(".")[0]))
files_2017 = names[names<=1944417].apply(lambda x: str(x)+".csv")
files_2018 = names[names>1944417].apply(lambda x: str(x)+".csv")
column = "得点"
teams_mlb = list(set(pd.read_csv(os.path.join(SCORE_DATA_DIR_MLB, "score_mlb.csv"), encoding="cp932", index_col=0).表チーム))
teams_mlb.remove("ア・リーグ")

In [ ]:
#pd.DataFrame(np.array([1, 2, np.nan])).fillna(-1).astype(int).replace(-1, np.nan).values.flatten()

In [ ]:
event_df.head(100)

In [3]:
event_set = set(event_df.状況)
len(event_set)

25

In [4]:
extracted_df = event_df[["状況", "裏得点", "表得点"]]
#extracted_df = extracted_df[extracted_df.状況!="3000"]
extracted_df.head()

,状況,裏得点,表得点
0,0000,0,0
1,1000,0,0
2,2000,0,0
3,2100,0,0
4,3000,0,0


In [5]:
event_dict = defaultdict(list)
extracted_array = extracted_df.values
for idx in range(len(extracted_array)-1):
    curr_case, curr_bot_score, curr_top_score = extracted_array[idx]
    late_case, late_bot_score, late_top_score = extracted_array[idx+1]
    try:
        curr_bot_score = int(curr_bot_score)
    except:
        pass
    try:
        curr_top_score = int(curr_top_score)
    except:
        pass
    try:
        late_bot_score = int(late_bot_score)
    except:
        pass
    try:
        late_top_score = int(late_top_score)
    except:
        pass
    event_dict[curr_case].append((late_bot_score+late_top_score)-(curr_bot_score+curr_top_score))
    """if curr_case == "2111":
        if True or (late_bot_score+late_top_score)-(curr_bot_score+curr_top_score) == 2:
            print(idx)
            break"""
event_dict = dict(event_dict)

In [ ]:
pd.Series(event_dict["2111"]).value_counts() #* pd.Series(event_dict["2111"]).value_counts().index

In [6]:
def expected_values(series):
    total = sum(series)
    if total == 0:
        return np.nan
    return sum(series.index * series) / total

In [7]:
mean_dict = dict()
for case in event_set:
    mean_dict[case] = expected_values(pd.Series(event_dict[case]).value_counts())

In [36]:
pd.DataFrame(pd.Series(mean_dict).sort_values().iloc[1:].astype(object))#.to_csv(os.path.join(ROOT_DIR, "case_expected_score.csv"), header=["expected_score"])

,0
0000,0.00268148
1000,0.0333274
2000,0.0335624
0100,0.0854814
1100,0.0881936
2100,0.0884837
0010,0.159629
1010,0.182915
2010,0.205981
0110,0.224608


In [ ]:
pd.Series([i for i in range(len(event_set))], index=pd.Series(mean_dict).sort_values().index).to_dict()

In [ ]:
d = dict({'0000': 1, '0001': 17, '0010': 8, '0011': 23, '0100': 5, '0101': 22, '0110': 11, '0111': 25, '1000': 2, '1001': 18, '1010': 9, '1011': 21, '1100': 6, \
 '1101': 19, '1110': 13, '1111': 24, '2000': 3, '2001': 12, '2010': 10, '2011': 16, '2100': 7, '2101': 15, '2110': 14, '2111': 20, '3000': 4, 'GAMESET': 0})

In [ ]:
d

In [ ]:
event_df.iloc[23:27, :]

In [ ]:
event_df.iloc[36236, :]